#   TASK1: Data Tagging

In [1]:
!pip uninstall -y numpy pandas
!pip install numpy pandas --upgrade --force-reinstall


Found existing installation: numpy 2.3.1
Uninstalling numpy-2.3.1:
  Successfully uninstalled numpy-2.3.1
Found existing installation: pandas 2.3.1
Uninstalling pandas-2.3.1:
  Successfully uninstalled pandas-2.3.1
  Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl (12.7 MB)
Using cached pandas-2.3.1-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
  Att

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.3.1 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.1 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.1 which is incompatible.
scipy 1.13.1 requires numpy<2.3,>=1.22.4, but you have numpy 2.3.1 which is incompatible.


In [8]:
!pip install openai

   ---------------------------------------- 0.0/755.6 kB ? eta -:--:--
   --------------------------------------- 755.6/755.6 kB 15.9 MB/s eta 0:00:00


In [14]:
df=pd.read_excel("C:/Users/HP/Downloads/DA - Task 1.xlsx")
df.head()

,Primary Key,Order Date,Product Category,Complaint,Cause,Correction,Root Cause,Symptom Condition 1,Symptom Component 1,Symptom Condition 2,Symptom Component 2,Symptom Condition 3,Symptom Component 3,Fix Condition 1,Fix Component 1,Fix Condition 2,Fix Component 2,Fix Condition 3,Fix Component 3
0,SO0026296-1,2023-03-08,SPRAYS,VISIBLY NOTICE fasteners under cab on P clips ...,Not tighten at factory.,"GO THROUGH AND RE-TIGHTEN ALL P CLIPS, NUTS, A...",Not Tightened,Loose,Cab P Clip,Loose,Left-Air Duct,Loose,Bulkhead Connector,Retightened,Cab P Clip,Retightened,Left Air Duct,Retightened,Bulkhead Connector
1,SO0026385-1,2023-03-08,SPRAYS,Fuel door will not stay open,GAS STRUT NOT INSTALLED OR ANYWHERE ON MACHINE,FOUND GAS STRUT NOT INSTALLED OR ANYWHERE ON M...,Not Installed,Won't stay open,Fuel Door,NaN,NaN,NaN,NaN,Installed,Gas Strut,NaN,NaN,NaN,NaN
2,SO0026385-11,2023-03-08,SPRAYS,"Compressor pressure line, braided steel, crushed","Compressor pressure line, braided steel, crush...",DRAIN AIR FROM SYSTEM.REMOVE ASSOCIATED P CLIP...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SO0028352-1,2023-03-08,SPRAYS,Oil running from bottom of machine,OIL RETURN UNDER MACHINE SWIVEL FITTING LEFT L...,OIL RETURN UNDER MACHINE SWIVEL FITTING LEFT L...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SO0028770-1,2023-03-08,SPRAYS,MISSING VECTOR & INTRIP UNLOCKS.,MISSING VECTOR & INTRIP UNLOCKS WERE NOT INSTA...,INSTALLED MISSING UNLOCKS RAN AND TESTED.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
import pandas as pd
import openai
import time


client = openai.OpenAI(api_key="sk-proj-t5...........")  #Openai-dia


#df = pd.read_csv("C:/Users/HP/Downloads/dia_task1.csv")
df = pd.read_excel("C:/Users/HP/Downloads/DA - Task 1.xlsx", sheet_name="Task")


# Step 3: Ensure required columns exist
required_cols = [
    'Root Cause',
    'Symptom Condition 1', 'Symptom Component 1',
    'Symptom Condition 2', 'Symptom Component 2',
    'Symptom Condition 3', 'Symptom Component 3',
    'Fix Condition 1', 'Fix Component 1',
    'Fix Condition 2', 'Fix Component 2',
    'Fix Condition 3', 'Fix Component 3'
]

for col in required_cols:
    if col not in df.columns:
        df[col] = ""

# === Step 4: GPT Extraction Function ===
def extract_labels(complaint, cause, correction):
    prompt = f"""
You are given a Complaint, its Cause, and the Correction steps performed. Your task is to extract the following structured information:

1. Root Cause → What caused the issue (e.g., Not Tightened, Not Installed)
2. Up to 3 Symptom Conditions and Components (e.g., "Loose" + "Cab P Clip")
3. Up to 3 Fix Conditions and Components (e.g., "Retightened" + "Cab P Clip")

Use this format exactly:
Root Cause: <text>
Symptom 1: <condition>, <component>
Symptom 2: <condition>, <component>
Symptom 3: <condition>, <component>
Fix 1: <condition>, <component>
Fix 2: <condition>, <component>
Fix 3: <condition>, <component>

If fewer than 3 symptoms or fixes are present, leave the rest blank.

Complaint: "{complaint}"
Cause: "{cause}"
Correction: "{correction}"
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4",  # or "gpt-3.5-turbo" for cheaper usage
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )

        content = response.choices[0].message.content.strip().splitlines()
        data = {col: "" for col in required_cols}

        for line in content:
            line = line.strip()
            if line.lower().startswith("root cause:"):
                data['Root Cause'] = line.split(":", 1)[1].strip()
            elif line.lower().startswith("symptom 1:"):
                parts = line.split(":", 1)[1].split(",", 1)
                if len(parts) == 2:
                    data['Symptom Condition 1'] = parts[0].strip()
                    data['Symptom Component 1'] = parts[1].strip()
            elif line.lower().startswith("symptom 2:"):
                parts = line.split(":", 1)[1].split(",", 1)
                if len(parts) == 2:
                    data['Symptom Condition 2'] = parts[0].strip()
                    data['Symptom Component 2'] = parts[1].strip()
            elif line.lower().startswith("symptom 3:"):
                parts = line.split(":", 1)[1].split(",", 1)
                if len(parts) == 2:
                    data['Symptom Condition 3'] = parts[0].strip()
                    data['Symptom Component 3'] = parts[1].strip()
            elif line.lower().startswith("fix 1:"):
                parts = line.split(":", 1)[1].split(",", 1)
                if len(parts) == 2:
                    data['Fix Condition 1'] = parts[0].strip()
                    data['Fix Component 1'] = parts[1].strip()
            elif line.lower().startswith("fix 2:"):
                parts = line.split(":", 1)[1].split(",", 1)
                if len(parts) == 2:
                    data['Fix Condition 2'] = parts[0].strip()
                    data['Fix Component 2'] = parts[1].strip()
            elif line.lower().startswith("fix 3:"):
                parts = line.split(":", 1)[1].split(",", 1)
                if len(parts) == 2:
                    data['Fix Condition 3'] = parts[0].strip()
                    data['Fix Component 3'] = parts[1].strip()

        return data

    except Exception as e:
        print(f" GPT error on row: {e}")
        return {col: "" for col in required_cols}

# Step 5: Process each row 
for idx, row in df.iterrows():
    print(f" Processing row {idx + 1}/{len(df)}")

    result = extract_labels(row['Complaint'], row['Cause'], row['Correction'])

    for col, val in result.items():
        df.at[idx, col] = val

    time.sleep(1.5)  # Respect API rate limits

# Step 6: Save Output
output_file = "C:/Users/HP/Downloads/Task_1 After_Tagging.csv"
df.to_csv(output_file, index=False)
print(f" All done! Results saved to '{output_file}'")


 Processing row 1/20
 Processing row 2/20
 Processing row 3/20
 Processing row 4/20
 Processing row 5/20
 Processing row 6/20
 Processing row 7/20
 Processing row 8/20
 Processing row 9/20
 Processing row 10/20
 Processing row 11/20
 Processing row 12/20
 Processing row 13/20
 Processing row 14/20
 Processing row 15/20
 Processing row 16/20
 Processing row 17/20
 Processing row 18/20
 Processing row 19/20
 Processing row 20/20
 All done! Results saved to 'C:/Users/HP/Downloads/Task_1 After_Tagging.csv'


### BY, DIYA HALDANKAR